# タイトル: SageMaker Training で最大級に高価な足し算と引き算を行って、Inference で仮想モデルをデプロイする🐱

## 本ハンズオンのお品書き
- 【前菜】SageMaker Training と Inference の全体像
- 【肉 1】SageMaker Training - 座学編
- 【肉 2】SageMaker Training - ハンズオン編
- 【魚 1】SageMaker Inference - 座学編
- 【魚 2】SageMaker Inference - ハンズオン編
- 【お持ち帰り用デザート（肉）】SageMaker Training と Inference の応用編

***※ SageMaker の機能を表現するために一部過剰な表現が含まれていることがあります。予めご了承ください。***

# 【前菜】SageMaker Training と Inference の全体像

![overview-training](./images/SageMaker-whole-image.png)

# 【肉 1】SageMaker Training - 座学編

## 全体のイメージ

![overview-training](./images/overview-training.png)

SageMaker Training の機能とは、ユーザーが用意した「コード」と「データ」を指定した「コンピューティングリソース」と「環境」で実行し、「実行履歴を自動記録」して、「アーティファクトを自動保存」する機能。

## 構成要素

* 学習データ
* 学習コード
* DL/ML 実行環境 (TensorFlow、PYTORCH、mxnet、Hugging Face、または scikit-learn など)

## SageMaker Python SDK
* SageMaker 上でモデルの学習やデプロイなどを行うためのオープンソースライブラリ
* データサイエンティストが簡単に AWS を使えるよう、SageMaker とその周辺のサービスに絞った高レベル API を提供

※ SageMaker 自体は CLI や Python SDK (Boto3) などからも利用可能

## SDK でトレーニングを開始する準備

![prepare-training](./images/prepare-training.png)

(必要に応じて) トレーニングデータとトレーニング用のコンテナイメージを用意し、それぞれ S3 と ECR に格納する。

## SDK でトレーニングの開始時の全体像

![start-training](./images/start-training.png)

* SDK の API 経由でトレーニングを定義して、トレーニングを開始する。
* 各フレームワークごとにジョブ定義用の estimator を利用して、使用するコードやバージョンを定義する。

## トレーニング完了時の全体像

![completed-training](./images/completed-training.png)

* トレーニング完了時にアーティファクト (コードやモデルなどの総称) が S3 に自動で転送される。
* ジョブ終了時に自動でインスタンスが削除され課金が停止する。
* 実行中のメトリクス/標準出力は CloudWatch に自動で転送される。

# 【肉 2】SageMaker Training - ハンズオン編

## 1-1. SageMaker SDK のバージョンを最新にしておく

In [1]:
pip install sagemaker -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 86.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.7
    Uninstalling dill-0.3.7:
      Successfully uninstalled dill-0.3.7
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.15
    Uninstalling multiprocess-0.70.15:
      Successfully uninstalled multiprocess-0.70.15
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.64
    Uninstalling boto

In [2]:
import sagemaker
print(sagemaker.__version__)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
2.210.0


## 1-2. データをトレーニングインスタンスに持ち込む
### 1-2-1. 単一のファイルを持ち込む
#### 持ち込むデータの確認
算数の計算問題を記載したデータを用意

In [3]:
with open('./data/1-2-1/calc.txt','rt') as f:
    input_text_lines = f.read()
print('---データの確認---')
print(input_text_lines)
print('---計算結果---')
for input_text in input_text_lines.split('\n'):
    print(eval(input_text))

---データの確認---
3+4
4-2
5*1
6/2
---計算結果---
7
2
5
3.0


#### 持ち込むデータを S3 にアップロード
`upload_data` メソッドを使うと S3 にデータをアップロードできる。返り値は S3 の URI

In [4]:
# S3 にデータをアップロード
import sagemaker

input_s3_uri = sagemaker.session.Session().upload_data(
    path='./data/1-2-1/calc.txt', 
    bucket=sagemaker.session.Session().default_bucket(), 
    key_prefix='training/1-2-1'
)
print(input_s3_uri)

s3://sagemaker-us-east-1-222426255091/training/1-2-1/calc.txt


#### 用意したコードを確認 (BYOA)
* トレーニングインスタンスに連携するデータはデフォルトだと環境変数 `SM_CHANNEL_TRAINING`(=`/opt/ml/input/training`) の値が示すディレクトリに格納される
* SM_CHANNEL_TRAINING にあるファイルを読み込み、1 行ずつデータを読み込んで文字列を数式として解釈して演算するコードを準備

In [5]:
!pygmentize ./src/1-2-1/calc.py

import os
input_dir = os.environ.get('SM_CHANNEL_TRAINING')
input_txt_path = os.path.join(input_dir,os.listdir(input_dir)[0])
with open(input_txt_path,'rt') as f:
    input_text_lines = f.read()
for input_text_line in input_text_lines.split('\n'):
    print(eval(input_text_line))
exit()


### Containers!!
AWS のマネジメントコンテナ(AWS Deep Learning Container / SageMaker Scikit-learn Container) で Training Job を実行する
各マネジメントコンテナ用のクラスが SageMaker SDK に準備されていて、その class をインスタンス化( `estimator` 変数でインスタンス化する)する際に引数でバージョンを指定することで利用するコンテナが確定する

|  class  |  コンテナ  | コンテナ詳細 | 
| ---- | ---- | ---- | 
|  [`sagemaker.tensorflow.TensorFlow`](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#sagemaker.tensorflow.estimator.TensorFlow)  |  TensorFlow | [利用可能なバージョン一覧及びソース](https://github.com/aws/deep-learning-containers/blob/master/available_images.md) | 
|  [`sagemaker.pytorch.PyTorch`](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/sagemaker.pytorch.html#sagemaker.pytorch.estimator.PyTorch)  |  PyTorch  |^|
|  [`sagemaker.pytorch.MXNet`](https://sagemaker.readthedocs.io/en/stable/frameworks/mxnet/sagemaker.mxnet.html#sagemaker.mxnet.estimator.MXNet)  |  MXNet  |^|
|  [`sagemaker.pytorch.HuggingFace`](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#sagemaker.huggingface.HuggingFace)  |  HuggingFace  |^|
|  [`sagemaker.sklearn.SKLearn`](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html)  |  scikit-learn  |[コンテナソース](https://github.com/aws/sagemaker-scikit-learn-container)|


### TensorFlow コンテナで実行
各フレームワーク(TensorFlow, PyTorch, MXNet, HuggingFace, scikit-learn) 毎に用意された estimator class で ジョブを定義し、estimator インスタンスを生成する。
* `entry_point` 引数に用意したコードを指定することで使える
* フレームワーク毎のクラス + py_version + framework_version + instance_type で使用するコンテナイメージが確定する。
  * 以下の場合は python3.8 が入った TensorFlow が 2.7.1 の CPU に最適化されたコンテナイメージ  
    763104351884.dkr.ecr.{REGION}.amazonaws.com/tensorflow-training:2.7.1-cpu-py38
  * estimator インスタンスの `training_image_uri` メソッドでコンテナイメージの URI を確認できる
* estimator 生成時に `image_uri` 引数を指定することで直接コンテナイメージを指定することもできる

In [6]:
# TensorFlow コンテナで Training Job (About 5 min)
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    entry_point='./src/1-2-1/calc.py',
    py_version='py38', 
    framework_version='2.7.1',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
print(f'トレーニングに使用するコンテナイメージは {estimator.training_image_uri()} です')
estimator.fit(input_s3_uri)

トレーニングに使用するコンテナイメージは 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.7.1-cpu-py38 です


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2024-02-29-00-15-13-545


2024-02-29 00:15:14 Starting - Starting the training job...
2024-02-29 00:15:30 Starting - Preparing the instances for training...
2024-02-29 00:16:00 Downloading - Downloading input data...
2024-02-29 00:16:15 Downloading - Downloading the training image...
2024-02-29 00:17:06 Training - Training image download completed. Training in progress...2024-02-29 00:17:16.683749: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2024-02-29 00:17:16.683931: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2024-02-29 00:17:16.711703: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2024-02-29 00:17:18,555 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2024-02-29 00:17:18,564 sagemaker-training-toolkit INFO     

### PyTorch コンテナで実行
他のフレームワークでも使い方は全く一緒

In [ ]:
# PyTorch コンテナで Training Job
from sagemaker.pytorch import PyTorch
estimator = PyTorch(
    entry_point='./src/1-2-1/calc.py',
    py_version='py38', 
    framework_version='1.9.1',
    instance_count=1,
    instance_type='ml.m5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit(input_s3_uri)

### MXNet コンテナで実行

In [ ]:
# MXNet コンテナで Training Job
from sagemaker.mxnet import MXNet
estimator = MXNet(
    entry_point='./src/1-2-1/calc.py',
    py_version='py37', 
    framework_version='1.8.0',
    instance_count=1,
    instance_type='ml.m4.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit(input_s3_uri)

### HuggingFace コンテナで実行

In [ ]:
# HuggingFace コンテナで　Training Job
# 注意 : GPU のクオータ緩和をしないと使えません
# from sagemaker.huggingface import HuggingFace
# estimator = HuggingFace(
#     entry_point='./src/1-2-1/calc.py',
#     py_version='py37', 
#     transformers_version='4.6.1',
#     tensorflow_version='2.4.1',
#     instance_count=1,
#     instance_type='ml.g4dn.xlarge',
#     role=sagemaker.get_execution_role()
# )
# estimator.fit(input_s3_uri)

### scikit-learn コンテナで実行

In [ ]:
# scikit-learn コンテナで Training Job
from sagemaker.sklearn import SKLearn
estimator = SKLearn(
    entry_point='./src/1-2-1/calc.py',
    py_version='py3', 
    framework_version='0.23-1',
    instance_count=1,
    instance_type='ml.c5.xlarge',
    role=sagemaker.get_execution_role()
)
estimator.fit(input_s3_uri)

### ジョブの実行結果確認
使用したコンテナイメージ URI や実行時間、使ったコードのありかなどが記録される

In [7]:
# ジョブ定義や実行結果を確認
estimator.latest_training_job.describe()

{'TrainingJobName': 'tensorflow-training-2024-02-29-00-15-13-545',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:222426255091:training-job/tensorflow-training-2024-02-29-00-15-13-545',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-222426255091/tensorflow-training-2024-02-29-00-15-13-545/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'model_dir': '"s3://sagemaker-us-east-1-222426255091/tensorflow-training-2024-02-29-00-15-13-545/model"',
  'sagemaker_container_log_level': '20',
  'sagemaker_job_name': '"tensorflow-training-2024-02-29-00-15-13-545"',
  'sagemaker_program': '"calc.py"',
  'sagemaker_region': '"us-east-1"',
  'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-222426255091/tensorflow-training-2024-02-29-00-15-13-545/source/sourcedir.tar.gz"'},
 'AlgorithmSpecification': {'TrainingImage': '763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.7.1-cpu-py38',
  'TrainingIn

#### 演習 1-2-1
ファイルを適宜書き換えて、トレーニングの定義および開始のコードを作成してみましょう！（instance_type は local でも構いませんし、ml. 系列のインスタンスタイプを指定しても構いません）

##### 書き換えたい方向け

In [ ]:
# text = """3+4
# 4-2
# 5*1
# 6/2"""
# with open('./data/1-2-1/calc.txt','wt') as f:
#     f.write(text)

In [ ]:
# トレーニングジョブの実行
import sagemaker
from sagemaker.tensorflow import TensorFlow

## 以下はサンプルなので適宜変更してください。時間が限られている関係で fit() は呼び出しません。
input_s3_uri = sagemaker.session.Session().upload_data(・・・)
estimator = TensorFlow(・・・)
# estimator.fit(・・・)

# 【魚 1】SageMaker Inference - 座学編

## 全体のイメージ

![inference-overview](./images/inference-overview.png)

## 推論コードのお作法

![code-rule-inference](./images/code-rule-inference.png)

# 【魚 2】SageMaker Inference - ハンズオン編
Training Job を実行後、例えば以下のコードでデプロイまで実行できてしまいますが、今回のハンズオンでは仮想モデルを作成して Inference の Realtime-Endpoint の作成を行っていきます。
```
input_s3_uri = sagemaker.session.Session().upload_data(・・・)
estimator = TensorFlow(・・・)
estimator.fit(・・・)
estimator.deploy(・・・)
```

## 準備

### モジュールのインポート、定数の設定、boto3 クライアントの設定、ロールの取得

In [9]:
import sagemaker
from typing import Final
from sagemaker.pytorch import PyTorchModel
from sagemaker.async_inference import AsyncInferenceConfig
import os, boto3, json, numpy as np
from io import BytesIO
from time import sleep
from uuid import uuid4
smr_client = boto3.client('sagemaker-runtime')
sm_client = boto3.client('sagemaker')
s3_client = boto3.client('s3')
endpoint_inservice_waiter = sm_client.get_waiter('endpoint_in_service')
role = sagemaker.get_execution_role()
region = sagemaker.Session().boto_region_name
bucket = sagemaker.Session().default_bucket()

### モデルと推論コードを保存するディレクトリを作成

In [10]:
model_dir = 'model'
!if [ -d ./{model_dir} ]; then rm -rf ./{model_dir}/;fi
!mkdir ./{model_dir}/

source_dir = 'source'
!if [ -d ./{source_dir} ]; then rm -rf ./{source_dir}/;fi
!mkdir ./{source_dir}/

### モデル相当のテキストファイルを `tar.gz` で固めて S3 にアップロードする
* SageMaker で推論する場合は機械学習のモデルを `model.tar.gz` に固めておく必要がある
    * SageMaker Training を使ってモデルを保存した場合は自動で tar.gz になるが、このハンズオンでは Training Job を使わないため、手動で tar.gz に固める
    * 機械学習のモデルと言ったが、用意したファイルを読み込むコードを書き、その読み込んだデータを使って処理を行うだけなので必ずしも機械学習のモデルである必要はない
    * このチュートリアルでは Hello my great machine learning model と書かれたテキストファイル(`my_model.txt`)を作成して、`tar.gz` に固める
* `model.tar.gz` を推論環境で使うには予め S3 にアップロードしておく必要がある

#### `my_model.txt` 作成

In [11]:
%%writefile ./{model_dir}/my_model.txt
Hello my great machine learning model

Writing ./model/my_model.txt


#### `my_model.txt` を `model.tar.gz` に固める

In [12]:
%cd {model_dir}
!tar zcvf model.tar.gz ./*
%cd ..

/home/sagemaker-user/SageMaker-training-inference-basic-handson/model
./my_model.txt
/home/sagemaker-user/SageMaker-training-inference-basic-handson


#### `model.tar.gz` を S3 にアップロード

In [13]:
model_s3_uri = sagemaker.session.Session().upload_data(
    f'./{model_dir}/model.tar.gz',
    key_prefix = 'hello_sagemaker_inference'
)
print(model_s3_uri)

s3://sagemaker-us-east-1-222426255091/hello_sagemaker_inference/model.tar.gz


### 推論コードを作成する
* 最低限 `model_fn` と `predict_fn` が必要
* `model_fn` は `model.tar.gz` に固めたモデルを読み込むコード
  * 第一引数に `model.tar.gz` を展開したディレクトリが入る
  * 返り値にモデルを返すと、`predict_fn` の第二引数に入れられる
* `predict_fn` は推論コード
  * 第一引数にリクエスト(推論したいデータ)が入る
  * 第二引数に model_fn の返り値が入る
  * 推論結果を返り値に入れる

In [14]:
%%writefile ./{source_dir}/inference.py
import os
def model_fn(model_dir):
    with open(os.path.join(model_dir,'my_model.txt')) as f:
        model = f.read()[:-1] # 改行を除外
    return model
def predict_fn(input_data, model):
    response = f'{model} for the {input_data}st time'
    return response

Writing ./source/inference.py


## SageMaker SDK でモデルをデプロイしてリアルタイム推論
### SageMaker SDK を用いてモデルをデプロイ
SageMaker SDK でモデルをデプロイするには、[Model](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model) クラスでモデルを定義する必要がある  
今回は AWS が管理・公開している PyTorch のコンテナを使うため、`Model` を継承した [PyTorchModel](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/sagemaker.pytorch.html#sagemaker.pytorch.model.PyTorchModel) クラスを使用する。  
`PyTorchModel` では、モデルにつける任意の名前、使用するモデルの S3 の URI、フレームワークや Python のバージョン、推論コードなどを指定する。
PyTorchModel でインスタンスを生成したら、[deploy](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy) メソッドでモデルをデプロイできる。デプロイ時はインスタンスタイプと台数、エンドポイントにつける任意の名前を設定する。

In [15]:
# 名前の設定
model_name = 'PyTorchModel'
endpoint_name = model_name + 'Endpoint'

In [16]:
# モデルとコンテナの指定 (About 5 min)
pytorch_model = PyTorchModel(
    name = model_name,
    model_data=model_s3_uri,
    role= role,
    framework_version = '1.11.0',
    py_version='py38',
    entry_point='inference.py',
    source_dir=f'./{source_dir}/'
)
pytorch_predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    enpoint_name=endpoint_name
)

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-222426255091/hello_sagemaker_inference/model.tar.gz), script artifact (./source/), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-222426255091/PyTorchModel/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: PyTorchModel
INFO:sagemaker:Creating endpoint-config with name PyTorchModel-2024-02-29-00-35-56-331
INFO:sagemaker:Creating endpoint with name PyTorchModel-2024-02-29-00-35-56-331


-------!

In [17]:
response = pytorch_predictor.predict(1)
print(response)

Hello my great machine learning model for the 1st time <class 'numpy.ndarray'>


In [19]:
# endpointとモデルを削除
pytorch_predictor.delete_endpoint()
pytorch_model.delete_model()

INFO:sagemaker:Deleting endpoint configuration with name: PyTorchModel-2024-02-29-00-35-56-331
INFO:sagemaker:Deleting endpoint with name: PyTorchModel-2024-02-29-00-35-56-331
INFO:sagemaker:Deleting model with name: PyTorchModel


## 全体 Questions!! (Endpoint のデプロイが完了するまで)

***Q1. 複数の S3 データソースをトレーニングインスタンスに持ち込む場合はどうすればいいの？***

A. `SM_CHANNEL_{CHANNEL 名}` という環境変数が示すディレクトリに配置される。  
CHANNEL 名についてはユーザが `fit` メソッドを呼ぶときに決められる。  
`SM_CHANNELS` という環境変数に CHANNEL 名一覧が格納されるので、どのような CHANNEL 名が来てもいいようにコード側で対応しておく。

【Traiing Job の定義と実行】
```
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(
    entry_point='./src/calc.py',
    py_version='py38', 
    framework_version='2.7.1',
    instance_count=1,
    instance_type='local',
    role=sagemaker.get_execution_role()
)
estimator.fit({
    'folder1_data':folder1_input_s3_uri,
    'folder2_data':folder2_input_s3_uri,
})
```

【calc.py】
```
import os, json
channels = os.environ.get('SM_CHANNELS')
channels_list = json.loads(channels)
for channel in channels_list:
    input_dir = os.environ.get('SM_CHANNEL_' + channel.upper())
    for file_name in os.listdir(input_dir):
        input_txt_path = os.path.join(input_dir,file_name)
        with open(input_txt_path,'rt') as f:
            input_text_lines = f.read()
        for input_text_line in input_text_lines.split('\n'):
            print(eval(input_text_line))
exit()
```

***Q2. SageMaker SDK じゃなくて Boto3 で Endpoint を作成できますか。***

A. もちろんできます。以下のモデル作成、エンドポイントコンフィグ作成、エンドポイント作成の順で実現可能です。
1. [create_model](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model) でモデルと推論環境（推論コードやコンテナイメージ、環境変数の設定）をパッケージ化した Model を作成する
2. [create_endpoint_config](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config) で使用する Model や推論に使うコンピューティングリソース（インスタンスタイプ、台数など）や負荷の配分を設定する
3. [create_endpoint](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint) で EndpointConfig で設定した内容をデプロイする
 
※ 推論エンドポイントに推論データをリクエストする環境 (AWS Lambda など) には入っていないことが多い（boto3でやることが多い）。また、推論エンドポイント立ち上げもパイプラインに組み込む際は SageMaker SDK を使わない環境もありえる。

# 【お持ち帰り用デザート（肉）】SageMaker Training と Inference の応用編
以下の Git repo をご覧ください。
- https://github.com/SeongHaedu/SageMaker-training-inference-handson